# P6 Starter - Time Series Analysis 

### Statistical Modeling to Deep Learning

##  Imports & Sanity Check (Do NOT Change)

In [ ]:
import numpy as np 
import pandas as pd 
import os
import gc
from tqdm.notebook import tqdm
import statsmodels.api as sm # PACF, ACF
from typing import Tuple, List

# Viz:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("future.no_silent_downcasting", True)

## Helper Utilities. Read the Fn names atleast so that you are not re-writing code

* **make_submission**: Helps you convert your predictions to competition submission ready files.
* **rmsle**: Implementation of the metric used to evaluate your score on the leaderboard.
* **lgbm_rmsle**: Definition that can be used to do train-val type training while printing metric scores.

In [ ]:
def make_submission(test_preds):
    """
    Args:
        test_preds: Your predictions from the model.

    NOTE: Your test_predictions should be in the same order as the test set. 
          This function does not take care of unsorted/shuffled predictions.
    """
    test = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/test.csv")
    submission_df = pd.DataFrame(columns = ["id", "sales"])
    submission_df.sales = test_preds
    submission_df.id = test.id.astype(int)
    submission_df.to_csv("submission.csv", index = False)


def rmsle(y_pred, y_true):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))


def lgbm_rmsle(preds, train_data):
    labels = train_data.get_label()
    rmsle_val = rmsle(preds, labels)
    return 'RMSLE', rmsle_val, False

## Load the data (Do NOT Change)

In [ ]:
#########################
# DO NOT CHANGE
#########################

train = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/test.csv")
stores = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/stores.csv")
#sub = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/sample_submission.csv")   
transactions = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/transactions.csv").sort_values(["store_nbr", "date"])
oil = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/oil.csv")
holidays = pd.read_csv("/kaggle/input/cs-639-p-5-time-series-forecasting/store-sales-time-series-forecasting/holidays_events.csv")

# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)
oil["date"] = pd.to_datetime(oil.date)
holidays["date"] = pd.to_datetime(holidays.date)

# Data types
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

# Process holidays and events
tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis = 1)
holidays = holidays._append(tr).reset_index(drop = True)

# Additional Holidays
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

# Bridge Holidays
holidays["description"] = holidays["description"].str.replace("Puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])
 
# Work Day Holidays, that is meant to payback the Bridge.
work_day = holidays[holidays.type == "Work Day"]  
holidays = holidays[holidays.type != "Work Day"]  

# Events are national
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis = 1).rename({"description":"events"}, axis = 1)
holidays = holidays[holidays.type != "Event"].drop("type", axis = 1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
events["events"] =np.where(events.events.str.contains("futbol"), "Futbol", events.events)

train.head(5)

## Section 1: EDA & Feature Engineering

### Q1 Left join transaction to train and then print the Spearman Correlation between Total Sales and Transactions.

In [ ]:
# TODO - q1
print("Spearman Correlation between Total Sales and Transactions:")

### Q2 Plot an 'ordinary least squares' trendline between transactions and sales to verify the spearman correlation value in Q1. [0.1 Points]

In [ ]:
# TODO - q2

### Q3 Plot these line charts in the notebook:

A) Transactions vs Date (all stores color coded in the same plot) 

B) Average monthly transactions

 C) Average Transactions on the days of the wee)


In [ ]:
# TODO - q3 - Plot A

In [ ]:
# TODO - q3 - Plot B

In [ ]:
# TODO - q3 - Plot C

### Q4 Use pandas' in-build (linear) interpolation to impute the missing oil values then overlay the imputed feature over the original.

Your new feature column should be called: `dcoilwtico_interpolated`

In [ ]:
# Interpolate. 

# Plot

### Q5 Again, left join oil on the dataframe above and report the spearman correlation between oil and sales and oil and transactions

In [ ]:
print("Correlation with Daily Oil Prices:")
# Find correlation with sales & transactions

### Q6 Report the top-3 highest negative correlations between oil and sales of a particular product family. Now think whether oil should be discarded as a feature?

Use sort_values

In [ ]:
# Calculate all correlations

# Report the top 3

### Q7. Which 2 features do you think fit the description of look-ahead data leakage? [0.1 Points]

In [ ]:
print("The 2 data leakage features are: <feature-1> and <feature-2>")

### Q8. One hot encode the holidays and events data and similarly left join it to the the main dataframe.

You just have to finish the one-hot encoder function definition for this one.

In [ ]:
def one_hot_encoder(df, nan_as_category=True) -> Tuple[pd.DataFrame, List[str]]:
    # One hot encoding (pandas can do it on 1 line!) 
    
    # Store the new columns in a list
    
    # Replace " " with "_" in column names.
    
    # Return the new dataframe and all the columns (as a list)
    pass

In [ ]:
#########################
# DO NOT CHANGE. 
# NOTE: Run this after you have implemented the one_hot_encoder function above.
#########################

d = pd.merge(train._append(test), stores)
d["store_nbr"] = d["store_nbr"].astype("int8")

# National Holidays & Events
d = pd.merge(d, national, how = "left")
# Regional
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
# Local
d = pd.merge(d, local, how = "left", on = ["date", "city"])

# Work Day: It will be removed when real work day colum created
d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")

events, events_cat = one_hot_encoder(events, nan_as_category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1, events["events_Dia_de_la_Madre"])
events = events.drop(239)

d = pd.merge(d, events, how = "left")
d[events_cat] = d[events_cat].fillna(0)

# New features
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)
d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  'Independencia de Cuenca', 'Independencia de Guayaquil', 'Independencia de Guayaquil', 'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)

he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

d[["family", "city", "state", "type"]] = d[["family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

In [ ]:
# d.tail(10)

In [ ]:
#########################
# DO NOT CHANGE
#########################
train = d[d.date < "2017-08-01"]
test = d[d.date >= "2017-08-01"]
test = test.drop(["sales"], axis=1)

train = pd.get_dummies(train, columns=train.select_dtypes(['object']).columns)
train = pd.get_dummies(train, columns=train.select_dtypes(['category']).columns)
for col in train.columns:
    if pd.api.types.is_numeric_dtype(train[col]):
        train[col] = train[col].astype('float32')

test = pd.get_dummies(test, columns=test.select_dtypes(['object']).columns)
test = pd.get_dummies(test, columns=test.select_dtypes(['category']).columns)
for col in test.columns:
    if pd.api.types.is_numeric_dtype(test[col]):
        test[col] = test[col].astype('float32')

In [ ]:
replace_dict = {}
for x in train.cols:
    if "family" in x:
        if " " in x:
            lst = x.split(" ")
            new_feature_name = ""
            for k in lst:
                new_feature_name += str(k)
                new_feature_name += "_"
            replace_dict[x] = new_feature_name[:-1]
        if "," in x: 
            lst = x.split(",")
            new_feature_name = ""
            for k in lst:
                new_feature_name += str(k)
                new_feature_name += "_"
            replace_dict[x] = new_feature_name[:-1]
        if "/" in x:
            lst = x.split(",")
            new_feature_name = ""
            for k in lst:
                new_feature_name += str(k)
                new_feature_name += "_"
            replace_dict[x] = new_feature_name[:-1]
        
    elif "state" in x:
        lst = x.split(",")
        new_feature_name = ""
        for k in lst:
            new_feature_name += str(k)
            new_feature_name += "_"
        replace_dict[x] = new_feature_name[:-1]

replace_dict

In [ ]:
train.rename(columns=replace_dict)
test.rename(columns=replace_dict)

In [ ]:
import re
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

## Section 2

### Q9. EMA

Forecast window should be >=15 days since the test set is 15 days. **For this question use 16 as the forecast window**

In [ ]:
# Train EMAs for each family per store (pandas has an inbuilt ema function!)

In [ ]:
# Make the predictions

# Use the make_submission utility function provided to save a submission CSV. 

# Submit to competition and note your RMSLE score somewhere for this model type.

# NOTE - 1: You still need to go on the right panel and click submit 
# (make_submission will NOT submit to competition -> It just makes a submission ready file)
# NOTE - 2: Ensure that you are not overwriting your submission.csv file in subsequent cells.

### Q10. Linear Regressor

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# 1. Create a dictionary to store all the models (1 model per family per store)
models = {}
# 2. Fit these models

In [ ]:
# 1. Look through each test record and find the corresponding model in models

# 2. Make the predictions on the test set.

# 3. Make the submission using the utility function `make_submission` provided 

# 4. Submit to competition and note your RMSLE score somewhere for this model type.

### Q11. PACF and ACF

Use lib sm 

(statsmodel.api is already imported as sm)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
# 1. Group by date and family

# 2. Plot 33x2 charts (ACF & PACF for each family)


### Q12. ADF Test -> ARIMA

In [ ]:
# 1. Groupby date, sales and onpromotion

# 2. Now get the sales time series

# 3. Find the autocorrelation of the time series you just store in step 2.

# Print autocorrelation
print("Autocorrelation:")

In [ ]:
# Plot ACF on sales time series

In [ ]:
# Plot the PACF
fig, ax = plt.subplots(figsize=(10, 6))

##########
# TODO: Your plot code goes here:
##########

##########
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')
plt.title('Partial Autocorrelation Function (PACF)')

plt.show()

#### Differencing technique
This process is meant to transform the time series data to stationary, as ARIMA model only works with stationary time series data.

In [ ]:
# 1. Compute and store the diff series

# 2. Drop NA or any other erroneous values.


In [ ]:
# Compute the autocorrelation (nlags = 20)

# Plot the autocorrelation chart (use plt.stem)
plt.figure(figsize=(10, 6))

############
# TODO: Your code goes here:
############

############

plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Chart')
plt.show()

### Augmented Dickey-Fuller (ADF) test

The Augmented Dickey-Fuller (ADF) test is a statistical test used to determine whether a time series is stationary or non-stationary. Stationarity is an important assumption in many time series analysis models.

The ADF test evaluates the null hypothesis that the time series has a unit root, indicating non-stationarity. The alternative hypothesis is that the time series is stationary.

When performing the ADF test, we obtain the ADF statistic and the p-value. The ADF statistic is a negative number and the more negative it is, the stronger the evidence against the null hypothesis. The p-value represents the probability of observing the ADF statistic or a more extreme value if the null hypothesis were true. A low p-value (below a chosen significance level, typically 0.05) indicates strong evidence against the null hypothesis and suggests that the time series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
# TODO - q12 
# 1. Perform the ADF test

# 2. Extract the test statistics and p-value

# 3. Print these values
print("ADF Statistic:")
print("p-value:")

The ADF statistic is (around) -11.4. This statistic is a negative value and is more negative than the critical values at common significance levels. This suggests strong evidence against the null hypothesis of a unit root, indicating that the time series is stationary.

The p-values (around)  i6.76e-2121, which is a very small value close to zero. Typically, if the p-value is below a chosen significance level (e.g., 0.05), it indicates strong evidence to reject the null hypothesis. In your case, the extremely small p-value suggests strong evidence against the presence of a unit root and supports the stationarity of the time series.

**TODO** Choose the right p, q and d values for your ARIMA model

In [ ]:
# TODO: Replace with appropriate p,d,q values for ARIMA
p_arima = None

d_arima = None

q_arima = None

In [ ]:
# 1. Get sales series as training data (np array with appropriate dtype)

# 2. Using statsmodel.tsa lib. Initialize an ARIMA model with the p,d,q params you defined. 

# 3. Fit the model


In [ ]:
# Print the post model fitting summary
# print(result.summary())

In [ ]:
# Make predictions & submit to competition using your best model

## Section 3

### Q13 Define a validation set. What will be the most appropriate time period for this validation set?

In [ ]:
# Get the val set:

### Q14. LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
# Process your data to the appropriate dtypes, vars, etc.

In [ ]:
# Use the lgb.Dataset method to intialize your dataset iterables.

# 1. Make one for the train set:

# 2. Make another for the val set you defined in Q13:


In [ ]:
# Fill the dict with appropriate params:
lgb_params = {'num_leaves': ,
              'learning_rate': ,
              'feature_fraction': ,
              'max_depth': ,
              'verbose': 20,
              'num_boost_round': ,
              'early_stopping_rounds': ,
              'nthread': -1}

In [ ]:
# Complete the model initialization/train params)
model = lgb.train(lgb_params, ... 

In [ ]:
# 1. Predict the sales value on your val set using the best_iteration recorded by the LGBM
# 2. Compute and print the RMSLE on this val set.

In [ ]:
# 1. Pre-process your test set to appropriate format.
# 2. Predict -> Save using make_submission -> Submit to competition
# 3. Note your RMSLE for LGBM

### Q15. CatBoost

In [ ]:
from catboost import Pool, CatBoostRegressor

In [ ]:
# Fill out missing params for catboost appropriately here:
catboost_params = {
    'iterations': ,           # Number of boosting rounds
    'learning_rate': ,        # Learning rate for gradient boosting
    'depth': ,                   # Depth of each tree
    'loss_function': 'RMSLE',      # Loss function (Root Mean Squared Error for regression)
    'eval_metric': 'RMSLE',        # Evaluation metric
    'random_seed': 42,            # Ensures reproducibility
    'early_stopping_rounds': ,  # Stops training if no improvement after 50 rounds
    'verbose': 100                # Prints training progress every 100 rounds
}

In [ ]:
# 1. Define the model

# 2. Fit


In [ ]:
# 3. Preprocess your test data appropriately

# 4. Make Predictions

In [ ]:
# 5. Use make_submission -> Submit to competition

# 6. Note your RMSLE for this model

### Q16. XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
# 1. Initialize model with random state = 42 to be consistent with CatBoost

# 2. Fit


In [ ]:
# 3. Make Predictions.

In [ ]:
# 4. make_submission -> Submit to competition 

# 5. Note your RMSLE 

### Q17. Optuna for automatic hyperparameter optimization

In [ ]:
import optuna
import time

In [ ]:
def objective_lgb(trial):
    # 1. Define the parameter search space
    
    # 2. Create datasets (train, val) for LightGBM

    # 3. Train the model

    # 4. Evaluate on the validation set

    # 5. Return the metric score
    
    pass

# Create Optuna study to minimize the objective function

start = time.time()
# 1. Create the optuna study and specify appropriate direction

# 2. Optimize (pay attention to recommended trials; 50 takes too long)

# 3. Get the best parameters

# 4. Print them.
# print("Best parameters:", best_params)

print("Took:", time.time() - start, "seconds")

In [ ]:
# Make a competition submission using these parameters
# Note these values.

In [ ]:
# Do the same for Catboost

In [ ]:
# Do the same for XGBoost

### Q18. Which out of the three Catboost vs LightGBM vs XGBoost provides the best score? Why do you think this model is more suited to this dataset/problem?

In [ ]:
print("<Your answer goes here>")

## Optional Extra Credit Section - Achieve the lowest score

### Cross Validation Strategies & Ensembling

In [ ]:
# 1. Try different Validation sets 
# 2. Try ensembling different methods used in this assignment together